# Module Imports

In [54]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model


In [55]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [56]:
## FULL DATAFRAME
df_raw = pd.read_csv("NS_2-Policy_Summary.csv")

## Drop extraneous data points
df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vz_d']>=2.75)
    ].index)

df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vx_d']>=1.0)
    ].index)

df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


# df_temp = df.query(f"landing_rate>={0.2}")
# df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df['My_d'] = df['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max moment
df = df.drop(df[(df['vz_d']<= 2.25) & (df['vx_d']<= 0.75)].index) # Drop corner with no successful landings
df = df.dropna()

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

In [57]:
df

,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,RREV_sig,G1_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
24,1.5,1.00,0,0.904762,4.46,9.88,0.00,0.04,4.839579,-2.819105,9.879842,-85.746968,0.190000,[0. 0.],[0. 0.],[1.38 6.47],[1.5 1.5]
25,1.5,1.00,1,0.875000,5.17,2.17,0.00,0.00,6.095857,-3.550857,2.170000,-99.677419,0.251429,[0. 0.],[0. 0.],[6.28 2.74],[1.5 1.5]
26,1.5,1.00,2,0.708333,4.83,8.71,0.02,0.03,5.072882,-2.954882,8.706471,-106.956846,0.183647,[0. 0.],[0. 0.],[3.32 3.14],[1.5 1.5]
27,1.5,1.00,3,0.809524,4.88,6.50,0.01,0.00,5.322000,-3.100059,6.497000,-114.793691,0.179294,[0. 0.],[0. 0.],[4.6 2.75],[1.5 1.5]
28,1.5,1.00,4,0.700000,4.22,8.38,0.00,0.00,4.756000,-2.770143,8.380143,-124.519349,0.205143,[0. 0.],[0. 0.],[2.01 4.42],[1.5 1.5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,4.0,2.75,1,0.714286,5.30,6.64,0.01,0.01,5.751733,-3.742467,6.643333,-130.760512,0.189467,[0. 0.],[0. 0.],[4.27 5.64],[1.5 1.5]
757,4.0,2.75,2,0.916667,5.03,6.53,0.06,0.25,5.593409,-3.649273,6.652909,-129.975835,0.193364,[0. 0.],[0. 0.],[1.58 6.99],[1.5 1.5]
758,4.0,2.75,3,0.913043,3.99,2.13,0.00,0.05,4.211286,-2.742381,2.133095,-137.563149,0.307619,[0. 0.],[0. 0.],[4.75 2.4 ],[1.5 1.5]
759,4.0,2.75,4,0.833333,5.35,6.77,0.00,0.00,5.739000,-3.747450,6.771000,-133.279109,0.180900,[0. 0.],[0. 0.],[1.55 6.12],[1.5 1.5]


# Landing Rate Data

In [58]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(df['vx_d'],df['vz_d'],df['landing_rate'])

ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.plot_trisurf(df_raw_avg['vx_d'],df_raw_avg['vz_d'],df_raw_avg['landing_rate','mean'],cmap = cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

from scipy.interpolate import griddata


X = df_raw_avg['vx_d']
Y = df_raw_avg['vz_d']
Z = df_raw_avg['landing_rate','mean']

xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)


surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [60]:
## RREV Threshold vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

ax.scatter(df['vx_d'],df['vz_d'],df['RREV_threshold'])


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:

## Avg RREV_thr vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],df_avg['RREV_threshold','mean'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV_threshold')

ax.set_title('Avg RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [62]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

df_temp = df.query(f"landing_rate>={0.1}")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.scatter(df_temp['vx_d'],df_temp['vz_d'],df_temp['impact_tdelta'])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

df_temp = df.query(f"landing_rate>={0.1}")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df_temp['RREV_trigger'],df_temp['landing_rate'],df_temp['impact_tdelta'],c=df_temp['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')


ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_rate')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs RREV (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [64]:
## Define Dataframe

df_temp = df.query(f"landing_rate>={0.1}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

pcm = ax.scatter(df['vx_d'],df['vz_d'],df['My_d'])



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Raw Data)')

## I have more data for vz = (1.5,2.75) than vz = (3.00,4.0) so My_d differences are likely not a trend, just sparse data




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [65]:
df_temp = df.query(f"impact_eul<={-60}")
# df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()



## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.scatter(df_temp['vx_d'],df_temp['vz_d'],-df_temp['impact_eul'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')


ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])

## Failed landings in corner
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
## CLEAN DATAFRAME
df_temp = df.query(f"impact_eul<={-60}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=180)
ax.plot_trisurf(df_temp_avg['vx_d'],df_temp_avg['vz_d'],-df_temp_avg['impact_eul','mean'],cmap=cmap)


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.0)
ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

ax.set_title('Avg Impact Angle vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Triang Masking Example

In [102]:
## CLEAN DATAFRAME
df_temp = df_raw.query(f"impact_eul<={-60}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

import matplotlib.tri as mtri
x = df_temp_avg['vx_d']
y = df_temp_avg['vz_d']
z = -df_temp_avg['impact_eul','mean']

triang = mtri.Triangulation(x,y)

isBad = np.where((x<1.0) & (y<2.5), True, False)

mask = np.any(isBad[triang.triangles],axis=1)
triang.set_mask(mask)



# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)

# ax.triplot(triang, c="#D3D3D3", marker='.', markerfacecolor="#DC143C",
#     markeredgecolor="black", markersize=10)

# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# plt.show()

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=180)
ax.plot_trisurf(triang,-df_temp_avg['impact_eul','mean'],cmap=cmap)


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.0)
ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

ax.set_title('Avg Impact Angle vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [67]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

ax.scatter(df_temp['RREV_threshold'],df_temp['My_d'],c=df_temp['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')

ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')
## As Vz increases RREV_trigger decreases to preserve time to contact. It also finds a host of My_d that work and they vary because of the robustness of the legs and the fact that landing success isn't deterministic.
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['vx_d'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(2,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('vx_d')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['landing_rate'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(3,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_Rate')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['vx_d'],df['My_d'],df['RREV_threshold'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
# ax.set_zlim(0,-10)

ax.set_xlabel('Vx_d')
ax.set_ylabel('OF_y_trigger')
ax.set_zlabel('RREV_threshold')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# POLICY FITTING

## RAW POLICY RELATION

In [72]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.8}")

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR > 80%)')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.5}")

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR > 50%)')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [75]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.6}")


X = df_reg1[['RREV_threshold','OF_y']]
Y = df_reg1['My_d']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(f"Equation: My_d = {intercept:.2f} + {coeffs[0]:.2f}*RREV + {coeffs[1]:.2f}*OF_y \n")


## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = intercept + coeffs[0]*df_reg1['RREV_threshold'] + coeffs[1]*df_reg1['OF_y']
ax.plot_trisurf(df_reg1['RREV_threshold'],df_reg1['OF_y'],My_reg,label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

## Regression Analysis

import statsmodels.api as ssm       # for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)               # to add constant value in the model
model= ssm.OLS(Y,X).fit()           # fitting the model
predictions= model.summary()        # summary of the model
predictions





Equation: My_d = -9.74 + 3.29*RREV + 0.26*OF_y 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   My_d   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     87.52
Date:                Sun, 07 Feb 2021   Prob (F-statistic):           6.44e-31
Time:                        08:43:42   Log-Likelihood:                -519.50
No. Observations:                 316   AIC:                             1045.
Df Residuals:                     313   BIC:                             1056.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -9.7356      1.106     -8.805      0.000     -11.911      -7.560
RREV_threshold     3.2855      0.254     12.919      0.000       2.785       3.786
OF_y               0.2565      0.050      5.105      0.000       0.158       0.355
==============================================================================
Omnibus:                        0.127   Durbin-Watson:                   1.835
Prob(Omnibus):                  0.939   Jarque-Bera (JB):                0.217
Skew:                          -0.040   Prob(JB):                        0.897
Kurtosis:                       2.899   Cond. No.                         94.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [76]:
# def r_squared(y, y_hat): #Manual R^2 value
#     y_bar = y.mean()
#     ss_tot = ((y-y_bar)**2).sum()
#     ss_res = ((y-y_hat)**2).sum()
#     return 1 - (ss_res/ss_tot)

# R2 = r_squared(df_reg['My_d'],My_reg)
# print(f"R^2: {R2:.3f}")

## Regression with x-based loss


In [77]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.6}")


X = df_reg1[['My_d','OF_y']]
Y = df_reg1['RREV_threshold']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(f"Equation: RREV_reg = {intercept:.2f} + {coeffs[0]:.2f}*My_d + {coeffs[1]:.2f}*OF_y \n")


## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


RREV_reg = intercept + coeffs[0]*df_reg1['My_d'] + coeffs[1]*df_reg1['OF_y']
ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=5)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['vz_d'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

## Regression Analysis

import statsmodels.api as ssm       # for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)               # to add constant value in the model
model= ssm.OLS(Y,X).fit()           # fitting the model
predictions= model.summary()        # summary of the model
predictions





Equation: RREV_reg = 3.84 + 0.11*My_d + -0.10*OF_y 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         RREV_threshold   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.561
Method:                 Least Squares   F-statistic:                     202.4
Date:                Sun, 07 Feb 2021   Prob (F-statistic):           3.86e-57
Time:                        08:43:44   Log-Likelihood:                 23.269
No. Observations:                 316   AIC:                            -40.54
Df Residuals:                     313   BIC:                            -29.27
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8399      0.045     84.993      0.000       3.751       3.929
My_d           0.1059      0.008     12.919      0.000       0.090       0.122
OF_y          -0.1013      0.007    -13.627      0.000      -0.116      -0.087
==============================================================================
Omnibus:                        3.289   Durbin-Watson:                   1.888
Prob(Omnibus):                  0.193   Jarque-Bera (JB):                3.067
Skew:                           0.175   Prob(JB):                        0.216
Kurtosis:                       3.332   Cond. No.                         22.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
## PLOT FULL DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_clean['RREV_threshold'],df_clean['OF_y'],df_clean['My_d'],c=df_clean['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.8}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


for ii in np.arange(1,0.2,-.2):
    df_reg1 = df.query(f"landing_rate<={ii} & landing_rate>{ii-.2} & RREV_threshold>{3.5}")
    
    X = df_reg1[['My_d','OF_y']]
    Y = df_reg1['RREV_threshold']

    reg2 = linear_model.LinearRegression(normalize=True)
    reg2.fit(X,Y)

    intercept = reg2.intercept_
    coeffs = reg2.coef_

    # print(intercept)
    # print(coeffs)

    RREV_reg = intercept + coeffs[0]*df_reg1['My_d'] + coeffs[1]*df_reg1['OF_y']
    ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression')

    R2 = r_squared(df_reg1['RREV_threshold'],RREV_reg)
    print(f"R^2: {R2:.3f}")

ax.set_xlim(6,3.5)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

R^2: 0.483
R^2: 0.598
R^2: 0.577
R^2: 0.625


In [80]:
import scipy.odr

## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.8}")


# X = df_reg1[['My_d','OF_y']]
x1 = df_reg1['RREV_threshold'].to_numpy()
x2 = df_reg1['OF_y'].to_numpy()
X = np.row_stack( (x1, x2) )

Y = df_reg1['My_d'].to_numpy()

def linfit(beta,x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2]

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(X,Y)
odrfit = scipy.odr.ODR(data,linmod,beta0 = [-0.1,0.1,4.0])
odrres = odrfit.run()
odrres.pprint()




## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = odrres.beta[0]*df_reg1['RREV_threshold'] + odrres.beta[1]*df_reg1['OF_y'] + odrres.beta[2]
ax.plot_trisurf(df_reg1['RREV_threshold'],df_reg1['OF_y'],My_reg,label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()


Beta: [  9.72104614   0.95543735 -37.01514254]
Beta Std Error: [1.00122426 0.16283671 4.25435866]
Beta Covariance: [[ 22.9145019    2.73908881 -96.83460866]
 [  2.73908881   0.60611124 -10.52538444]
 [-96.83460866 -10.52538444 413.72893526]]
Residual Variance: 0.04374740579426504
Inverse Condition #: 0.003438928922915538
Reason(s) for Halting:
  Sum of squares convergence


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …